In [1]:
# Here we take care of paths.
# Make sure root project directory is named 'VESUVIUS_Challenge' for this to work

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in the root folder of the project
print('Current path:' + os.getcwd())

Starting path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge/jupyter notebooks
Current path:/Users/gregory/PROJECT_ML/VESUVIUS_Challenge


In [2]:
import torch
import monai
#from monai.visualize import matshow3d
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import Tuple, List
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Dataset import Vesuvius_Tile_Datamodule
from lit_models.Vesuvius_Lit_Model import Lit_Model
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn as nn
import torch.nn as nn
from functools import partial
import torchvision
import torch.nn.functional as F
from lit_models.scratch_models import FPNDecoder
from Models.PreBackbone_3D import PreBackbone_3D
from Models.PreBackbone_3d_Zdim import PreBackbone_3D_ZDIM
from Models.Segformer import SegFormer
from einops import rearrange
from Models.Model_3d import Encoder_3D

2023-05-25 10:33:42,163 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpzjlfj_5z
2023-05-25 10:33:42,163 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmpzjlfj_5z/_remote_module_non_scriptable.py


In [3]:
PATCH_SIZE = 256
Z_DIM = 12
COMPETITION_DATA_DIR_str =  "kaggle/input/vesuvius-challenge-ink-detection/"


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

# change to the line below if not using Apple's M1 or chips
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
encoder =PreBackbone_3D_ZDIM (out_channels = 6, z_dim= 12,att_dim=64,)




In [5]:
dummy = torch.randn(5,12,256,256)
out = encoder(dummy)
print(out.shape)

torch.Size([5, 12, 1024]) torch.Size([5, 12, 64])
torch.Size([5, 6, 256, 256])


In [6]:
pool(out).shape2

NameError: name 'pool' is not defined

In [ ]:
y = torch.randn(5,128,)

In [ ]:
outs= torch.nn.functional.avg_pool3d(out, kernel_size=(out.shape[2], 1, 1))

In [ ]:
outs.shap

In [ ]:
Pre_model3d = PreBackbone_3D_ZDIM(z_dim=16, out_channels = 8)
segformer = SegFormer(in_channels=3,
    widths=[64, 128, 256, 512],
    depths=[3, 4, 6, 3],
    all_num_heads=[1, 2, 4, 8],
    patch_sizes=[7, 3, 3, 3],
    overlap_sizes=[4, 2, 2, 2],
    reduction_ratios=[8, 4, 2, 1],
    mlp_expansions=[4, 4, 4, 4],
    decoder_channels=256,
    scale_factors=[8, 4, 2, 1],
    num_classes=1,
                    )

In [ ]:
dummy = torch.randn(16, 16,256,256)

In [ ]:
outs = Pre_model3d(dummy)

In [ ]:
outs.shape

In [ ]:
u

In [ ]:
Pre_model3d = PreBackbone_3D_ZDIM(z_dim=24)
dummy = torch.randn(16, 24,256,256)
outs = Pre_model3d(dummy)
print(outs.shape)

In [ ]:
segformer = SegFormer(
    in_channels=3,
    widths=[64, 128, 256, 512],
    depths=[3, 4, 6, 3],
    all_num_heads=[1, 2, 4, 8],
    patch_sizes=[7, 3, 3, 3],
    overlap_sizes=[4, 2, 2, 2],
    reduction_ratios=[8, 4, 2, 1],
    mlp_expansions=[4, 4, 4, 4],
    decoder_channels=256,
    scale_factors=[8, 4, 2, 1],
    num_classes=1,
)

In [ ]:
out_final  = segformer(outs)

In [ ]:
out_final.shape

In [ ]:
final = nn.functional.interpolate(
            out_final, 
            size=(256,256), 
            mode="bilinear", 
            align_corners=False)

In [ ]:
final.shape

In [ ]:
u

# 3d Convolutions

#### to decrease z_dim/channels

In [ ]:
import torch
import torch.nn as nn


# TODO: implement attnetion between slices. currently only local info


# TODO: try using Depth-wise convolution https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec
# TODO:
class PreBackbone_3D(nn.Module):
    def __init__(self, filter_sizes = [6,12,24,48], batch_norm=False):


        super(PreBackbone_3D, self).__init__()

        self.leaky_relu = nn.LeakyReLU(inplace=True)
        self.batch_norm = batch_norm
        self.pool = nn.AvgPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1))
        self.global_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.batch_norm = torch.nn.BatchNorm3d(num_features=1, momentum=0.9)

        # layer 1
        self.conv1 = nn.Conv3d(in_channels=1,
                               out_channels=filter_sizes[0],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )

        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)

        # layer 2
        self.conv2 = nn.Conv3d(in_channels=filter_sizes[0]//2,
                               out_channels=filter_sizes[1],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)

        # layer 3
        self.conv3 = nn.Conv3d(in_channels=filter_sizes[1]//2,
                               out_channels=filter_sizes[2],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )

        torch.nn.init.xavier_uniform_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)

        # layer 4
        self.conv4 = nn.Conv3d(in_channels=filter_sizes[2]//2,
                               out_channels=filter_sizes[3],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )

        torch.nn.init.xavier_uniform_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)






    def forward(self, x):
        x = x.unsqueeze(1)  # (B,C,H,W) -> (B, 1,  C, H, W)

        # Layer 1
        y = self.conv1(x)   # (B, 1,  C, H, W) -> (B, 4,  C/2, H, W)

        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)                # (B, 4,  C/2, H, W) ->  (B, 2,  C/2, H, W)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)
        # if self.batch_norm:
        #   y = self.batch_norm(y)

        # Layer 2
        y = self.conv2(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)

        # Layer 3
        y = self.conv3(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)

        # Layer 4
        y = self.conv4(y)  #(B, 48,  3, H, W)  where filter_sizes[-1]=48

       # Final convolution
        y = y.permute(0, 2, 1, 3, 4)
        y = self.global_pool(y)
        y = y.permute(0, 2, 1, 3, 4)

        y = self.leaky_relu(y)


        y = self.batch_norm(y)
        return y.squeeze(1)



In [ ]:
dummy = torch.randn(5,64,256,256)
pre_model = PreBackbone_3D()
pre_out = pre_model(dummy)
print(pre_out.shape)

# IDEA:

- take 48 slices and embed them and apply attention
- use residual connection at the end

In [ ]:
class LayerNorm2d(nn.LayerNorm):
    def forward(self, x):
        x = rearrange(x, "b c d h w -> b d h w c")
        x = super().forward(x)
        x = rearrange(x, "b d h w c -> b c d h w")
        return x
    
    
    
    
    
class Embed(nn.Module):
    def __init__(self, in_channels=1, emdedding_dims = emdedding_dims,):
        super().__init__()
        
        self.conv_3d =  nn.Conv3d(in_channels=1,
                               out_channels=emdedding_dims[0],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )
    

        self.conv_3d_embed =  nn.Conv3d(in_channels=emdedding_dims[0],
                               out_channels=1,
                               kernel_size=(1, 4, 4),
                               stride=(1, 4, 4),
                               padding=(0, 1, 1)
                               )
    
        self.norm =  LayerNorm2d(emdedding_dims[0])
        self.norm_embed =LayerNorm2d(1)
        
        
        
    
    def forward(self, x):
        x = self.conv_3d(x)
        x = self.norm(x)
        x_embed = self.conv_3d_embed(x)
        x_embed = self.norm_embed(x_embed)
        return x, x_embed 
    
    
    
    
    
    
    
    
class EfficientMultiHeadAttention(nn.Module):
    def __init__(self, channels= Z_DIM//2, reduction_ratio: int = 1, num_heads: int = 8):
        super().__init__()
        
        self.reducer = nn.Sequential(
            nn.Conv2d(in_channels= channels,
                               out_channels= channels,
                               kernel_size=( 4, 4),
                               stride=( 4, 4),
                               padding=( 0, 0)),
            
                            LayerNorm_att(channels), )
        
        
        self.att = nn.MultiheadAttention(PATCH_SIZE, num_heads=8, batch_first=True)
        
      
        
        

    def forward(self, x,):
        x = x.squeeze(1) #  (b z h w)
        
       
        reduced_x = self.reducer(x)
        
        _, c, h, w = reduced_x.shape 
        
        # attention needs tensor of shape (batch, sequence_length, channels)
        reduced_x = rearrange(reduced_x, "b c h w -> b  c ( h w )")
        x = rearrange(x, "b c  h w -> b  c ( h w)")
        out = self.att(reduced_x, reduced_x, reduced_x)[0]
        # reshape it back to (batch, channels, height, width)
        out = rearrange(out, "b  c (h w) -> b c h w", h=h, w=w,)
        out = nn.functional.interpolate( out, size=(256,256),  mode="bilinear", align_corners=False )
        out = out.unsqueeze(1)
        
        return out
    
    
    
class LayerNorm_att(nn.LayerNorm):
    def forward(self, x):
        x = rearrange(x, "b c h w -> b  h w c")
        x = super().forward(x)
        x = rearrange(x, "b  h w c -> b c  h w")
        return x   


In [ ]:
import torch
import torch.nn as nn


# TODO: implement attnetion between slices. currently only local info


# TODO: try using Depth-wise convolution https://medium.com/@zurister/depth-wise-convolution-and-depth-wise-separable-convolution-37346565d4ec
# TODO:


class PreBackbone_3D(nn.Module):
    def __init__(self, emdedding_dims = [4], filter_sizes=[16,32,48], batch_norm=False):


        super(PreBackbone_3D, self).__init__()
        
        self.embed_layer = Embed(emdedding_dims = emdedding_dims)
        self.attention = EfficientMultiHeadAttention()
        
        
        self.pool = nn.AvgPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1))
        self.global_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.batch_norm = torch.nn.BatchNorm3d(num_features=1, momentum=0.1)
        self.leaky_relu = nn.LeakyReLU(inplace=True)
        
        # layer 1
        self.conv1 = nn.Conv3d(in_channels=5,
                               out_channels=filter_sizes[0],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )

        torch.nn.init.xavier_uniform_(self.conv1.weight)
        torch.nn.init.zeros_(self.conv1.bias)

        # layer 2
        self.conv2 = nn.Conv3d(in_channels=filter_sizes[0]//2,
                               out_channels=filter_sizes[1],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        torch.nn.init.zeros_(self.conv2.bias)

        # layer 3
        self.conv3 = nn.Conv3d(in_channels=filter_sizes[1]//2,
                               out_channels=filter_sizes[2],
                               kernel_size=(3, 1, 1),
                               stride=(2, 1, 1),
                               padding=(1, 0, 0)
                               )

        torch.nn.init.xavier_uniform_(self.conv3.weight)
        torch.nn.init.zeros_(self.conv3.bias)
        
        
        

    def forward(self, x):
        
        # Stage 1 Embedding and Z_Dim attention
        
        # embed layer produces tensors:
        # x_orig = (B, emdedding_dims[0], C/2, H, W) for residual connection
        # x_att = (B, 1, C/2, H/4, W/4)
        x_orig, x_att = self.embed_layer(x)
        
        # attention layer for z_dim
        x_after_att = self.attention(x_att)
        
        x = torch.cat((x_orig,x_after_att ), dim=1)
        
        # Stage 2 Convolutions -- 3 layers
        
        # Layer 1
        y = self.conv1(x)   # (B, 1,  C, H, W) -> (B, 4,  C/2, H, W)

        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)                # (B, 4,  C/2, H, W) ->  (B, 2,  C/2, H, W)
    
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)
        # if self.batch_norm:
        #   y = self.batch_norm(y)

        # Layer 2
        y = self.conv2(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.pool(y)
        y = y.permute(0, 2, 1, 3, 4)
        y = self.leaky_relu(y)

        # Layer 3
        y = self.conv3(y)
        
        # Final convolution
        y = y.permute(0, 2, 1, 3, 4)
        y = self.global_pool(y)
        y = y.permute(0, 2, 1, 3, 4)

        y = self.leaky_relu(y)


        y = self.batch_norm(y)
        return y.squeeze(1)
        

        
        
    
    
    
    
    


In [ ]:
dummy = torch.randn(5,1,48,256,256)
pre_model = PreBackbone_3D()
out = pre_model(dummy)
print(out.shape)

# WORK TEST

#### STEP 1
outputs 2 tensors (for res connection and smaller for attention)
- `Embedding (B,1, C, H, W) -> (B, emdedding_dims[0], C/2, H, W)`
-

In [ ]:
# embed the input by convoluting in z-dim: z_dim/2

dummy = torch.randn(5,1,48,256,256)

embed_layer = Embed()
dumm_or, dummy_embed = embed_layer(dummy)
print(dumm_or.shape, dummy_embed.shape)

### STEP 2
 takes `dummy_embed` and applies attention
- Attention
- channles must be last
- reduce further and then add back the original tensor

print('initial shape:', dummy_embed.shape)
att = EfficientMultiHeadAttention()



dummy_reduced = att.reducer(dummy_embed.squeeze())
print('reduced shape:',dummy_reduced.shape)

#dummy_att = att.att(dummy_embed,dummy_reduced, dummy_reduced )




In [ ]:
# FULL MODULE
att = EfficientMultiHeadAttention()
dummy_after_att = att(dummy_embed)
print(dummy_after_att.shape)

# Attention

- channles must be last

In [ ]:
att_out = att(conv_out)

In [ ]:
class PVT_w_FPN(nn.Module):
    def __init__(self, in_channels,  embed_dims=[  64, 128, 256, 512], n_classes=1, ):
        super().__init__()
        
        self.embed_dims = embed_dims
        
       
        self.pvt = PyramidVisionTransformerV2(img_size = PATCH_SIZE,
                                  patch_size = 4,
                                  in_chans = Z_DIM,
                                  num_classes = 1,
                                  embed_dims = embed_dims,
                                num_heads=[1, 2, 4, 8],
                                  mlp_ratios=[4, 4, 4, 4],
                                  qkv_bias=True,
                                  qk_scale=None,
                                  drop_rate=0.,
                                attn_drop_rate=0.,
                                  drop_path_rate=0.1,
                                  norm_layer=partial(nn.LayerNorm, eps=1e-3),
                                #norm_layer=nn.LayerNorm,          
                                  depths=[2, 2, 2,2],
                                  sr_ratios=[1, 1, 1, 1]
                                 ).to(DEVICE) 
        
        self.FPN = FPNDecoder(
                            in_channels = Z_DIM,
                            encoder_channels = embed_dims ,
                            encoder_depth=5,
                            pyramid_channels=256,
                            segmentation_channels=128,
                            dropout=0.2,
                            merge_policy="cat",).to(DEVICE) 
        
       

    def forward(self, x):
        #x = x.unsqueeze(1)
        #x = self.pre_model3d(x)
        #x = x.squeeze(1)
        
        pvt_outs = self.pvt(x)
        
        logits = self.FPN(*pvt_outs)
        
       
       
            
        
        return logits












dummy = torch.randn(5,8,256,256).to(DEVICE) 
model = PVT_w_FPN(in_channels =8 ,  embed_dims=[ 64, 128, 256, 512])
out = model(dummy)
print(out.shape)

# BACKBONE

In [ ]:
model = PVT_w_FPN(in_channels =8 ,  embed_dims=[ 64, 128, 256, 512])
dummy = torch.randn(5,8,256,256).to(DEVICE) 
pvt_outs = model.pvt(dummy)
print('pvt outputs')
for t in pvt_outs:
    print(t.shape)

# NECK FPN

In [ ]:
fpn_outs = model.FPN(*pvt_outs)
print(fpn_outs.shape)

In [ ]:
from lit_models.Vesuvius_Lit_Model import dice_coef_torch

In [ ]:
dummy_y = torch.ones(5,1,256,256)
loss = dice_coef_torch(fpn_outs, dummy_y)

In [ ]:
loss

# MLP HEAD

In [ ]:
class MLP(nn.Module):
    """
    Linear Embedding
    """
    def __init__(self, input_dim=2048, embed_dim=768):
        super().__init__()
        self.proj = nn.Linear(input_dim, embed_dim)

    def forward(self, x):
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x



class SegFormerHead(nn.Module):
    """
    SegFormer: Simple and Efficient Design for Semantic Segmentation with Transformers
    """
    def __init__(self, z_dim, in_channels, embedding_dim, dropout= 0, feature_strides=None, **kwargs):
        super().__init__()
        self.in_channels = in_channels
        self.feature_strides = feature_strides
        self.num_classes = 1
        self.dropout = dropout


        #decoder_params = kwargs['decoder_params']
        self.embedding_dim = embedding_dim

        self.linear_c4 = MLP(input_dim=self.in_channels[-1], embed_dim=self.embedding_dim)
        self.linear_c3 = MLP(input_dim=self.in_channels[-2], embed_dim=self.embedding_dim)
        self.linear_c2 = MLP(input_dim=self.in_channels[-3], embed_dim=self.embedding_dim)
        self.linear_c1 = MLP(input_dim=self.in_channels[-4], embed_dim=self.embedding_dim)
        self.linear_c0 = MLP(input_dim=z_dim, embed_dim=self.embedding_dim)

        self.conv_fuse = nn.Sequential(
                    nn.ConvTranspose2d(
                        embedding_dim*5, embedding_dim, kernel_size=1, stride=1),
                    torch.nn.SyncBatchNorm(embedding_dim, eps=1e-04, momentum=0.1),
                    #nn.GroupNorm(32, segmentation_channels, eps=1e-03),
                    nn.GELU(),
                    nn.ConvTranspose2d(
                        embedding_dim, embedding_dim, kernel_size=1, stride=1),
                ).to(DEVICE)

        self.linear_pred = nn.Conv2d(embedding_dim, self.num_classes, kernel_size=1)
        self.dropout = nn.Dropout2d(p=self.dropout, inplace=True)

    def forward(self, *features):
        #x = self._transform_inputs(inputs)  # len=4, 1/4,1/8,1/16,1/32
        c0, c1, c2, c3, c4,  = features
        
        print(c0.shape,c1.shape, c2.shape, c3.shape, c4.shape)

        ############## MLP decoder on C1-C4 ###########
        n, _, h, w = c4.shape

        _c4 = self.linear_c4(c4).permute(0,2,1).reshape(n, -1, c4.shape[2], c4.shape[3])
        #_c4 = resize(_c4, size=c1.size()[2:],mode='bilinear',align_corners=False)
        _c4 =  F.interpolate(_c4, size=c0.size()[2:], scale_factor=None, mode='bilinear',align_corners=False)

        _c3 = self.linear_c3(c3).permute(0,2,1).reshape(n, -1, c3.shape[2], c3.shape[3])
        #_c3 = resize(_c3, size=c1.size()[2:],mode='bilinear',align_corners=False)
        _c3  =  F.interpolate(_c3, size=c0.size()[2:], scale_factor=None, mode='bilinear',align_corners=False)

        _c2 = self.linear_c2(c2).permute(0,2,1).reshape(n, -1, c2.shape[2], c2.shape[3])
        #_c2 = resize(_c2, size=c1.size()[2:],mode='bilinear',align_corners=False)
        _c2 = F.interpolate(_c2, size=c0.size()[2:], scale_factor=None, mode='bilinear',align_corners=False)

        _c1 = self.linear_c1(c1).permute(0,2,1).reshape(n, -1, c1.shape[2], c1.shape[3])
        _c1 = F.interpolate(_c1 , size=c0.size()[2:], scale_factor=None, mode='bilinear',align_corners=False)
        
        _c0 =  self.linear_c0(c0).permute(0,2,1).reshape(n, -1, c0.shape[2], c0.shape[3])
        
        print('one', _c0.shape, _c1.shape, _c2.shape, _c3.shape, _c4.shape)
        
        cc =  torch.cat([_c4, _c3, _c2, _c1, _c0], dim=1)
        print(cc.shape)

        _c = self.conv_fuse(cc)

        x = self.dropout(_c)
        x = self.linear_pred(x)

        return x

In [ ]:
S_Head = SegFormerHead( z_dim = 8, in_channels =[  64, 128, 256, 512] , embedding_dim=128 ).to(DEVICE)

In [ ]:
mlp_out = S_Head(*pvt_outs)

In [ ]:
print(mlp_out.shape)

In [ ]:
mlp_out.shape

# SEGFORMER TEST

In [ ]:
seg_model = SegFormer( in_channels=8,
                        widths=[64, 128, 256, 512],
                        depths=[3, 4, 6, 3],
                        all_num_heads=[1, 2, 4, 8],
                        patch_sizes=[7, 3, 3, 3],
                        overlap_sizes=[4, 2, 2, 2],
                        reduction_ratios=[8, 4, 2, 1],
                        mlp_expansions=[4, 4, 4, 4],
                        decoder_channels=256,
                        scale_factors=[8, 4, 2, 1],
                        num_classes=1, )

In [ ]:
dummy = torch.randn(5,8,256,256)

In [ ]:
seg_out = seg_model(dummy)
final_out = nn.functional.interpolate(
            seg_out, 
            size=(256,256), 
            mode="bilinear", 
            align_corners=False
        )

In [ ]:
final_out.shape

In [ ]:
class PreBackbone_3D_old(nn.Module):
    def __init__(self, batch_norm = True ):
        
        
        
        super(PreBackbone_3D, self).__init__()
        
        self.leaky_relu = nn.LeakyReLU( inplace=True)
        self.batch_norm = batch_norm
        
        self.conv = nn.Conv3d(in_channels=1,
                            out_channels=1,
                             kernel_size = (3, 1, 1),
                             stride=(1, 1, 1),
                             padding= (1, 0, 0)
                             )
        
        torch.nn.init.xavier_uniform_(self.conv.weight)
        torch.nn.init.zeros_(self.conv.bias)
         
        self.pool = nn.AvgPool3d(kernel_size = (2,1,1), stride=(2,1,1))
        self.batch_norm = torch.nn.BatchNorm3d( num_features=1, momentum=0.9)
        
        
        
        
    

    def forward(self, x):
        x = x.unsqueeze(1) # (B,C,H,W) -> (B, 1 C, H, W)
    
        
        y = self.conv(x)
        y = self.pool(y)
        y = self.leaky_relu(y)
        if self.batch_norm:
            y = self.batch_norm(y)  # (B, 1 C, H, W) -> (B, 1 C/2, H, W)
            
       
        y = self.conv(y)
        y = self.pool(y)
        y = self.leaky_relu(y)
        if self.batch_norm:
            y = self.batch_norm(y)  # (B, 1 C, H, W) -> (B, 1 C/2, H, W)
            
            
        y = self.conv(y)
        y = self.pool(y)
        y = self.leaky_relu(y)
        if self.batch_norm:
            y = self.batch_norm(y)  # (B, 1 C, H, W) -> (B, 1 C/2, H, W)
            
        y = self.conv(y)
        y = self.pool(y)
        y = self.leaky_relu(y)
        if self.batch_norm:
            y = self.batch_norm(y)  # (B, 1 C, H, W) -> (B, 1 C/2, H, W)
            
        return y
      
